In [2]:
import kaggle
import pandas as pd

In [3]:
df = pd.read_csv("Data/LeagueofLegends.csv")

In [4]:
df.head()

,League,Year,Season,Type,blueTeamTag,bResult,rResult,redTeamTag,gamelength,golddiff,...,redMiddleChamp,goldredMiddle,redADC,redADCChamp,goldredADC,redSupport,redSupportChamp,goldredSupport,redBans,Address
0,NALCS,2015,Spring,Season,TSM,1,0,C9,40,"[0, 0, -14, -65, -268, -431, -488, -789, -494,...",...,Fizz,"[475, 475, 552, 842, 1178, 1378, 1635, 1949, 2...",Sneaky,Sivir,"[475, 475, 532, 762, 1097, 1469, 1726, 2112, 2...",LemonNation,Thresh,"[515, 515, 577, 722, 911, 1042, 1194, 1370, 14...","['Tristana', 'Leblanc', 'Nidalee']",http://matchhistory.na.leagueoflegends.com/en/...
1,NALCS,2015,Spring,Season,CST,0,1,DIG,38,"[0, 0, -26, -18, 147, 237, -152, 18, 88, -242,...",...,Azir,"[475, 475, 552, 786, 1097, 1389, 1660, 1955, 2...",CoreJJ,Corki,"[475, 475, 532, 868, 1220, 1445, 1732, 1979, 2...",KiWiKiD,Annie,"[515, 515, 583, 752, 900, 1066, 1236, 1417, 15...","['RekSai', 'Janna', 'Leblanc']",http://matchhistory.na.leagueoflegends.com/en/...
2,NALCS,2015,Spring,Season,WFX,1,0,GV,40,"[0, 0, 10, -60, 34, 37, 589, 1064, 1258, 913, ...",...,Azir,"[475, 475, 533, 801, 1006, 1233, 1385, 1720, 1...",Cop,Corki,"[475, 475, 533, 781, 1085, 1398, 1782, 1957, 2...",BunnyFuFuu,Janna,"[515, 515, 584, 721, 858, 1002, 1168, 1303, 14...","['Leblanc', 'Zed', 'RekSai']",http://matchhistory.na.leagueoflegends.com/en/...
3,NALCS,2015,Spring,Season,TIP,0,1,TL,41,"[0, 0, -15, 25, 228, -6, -243, 175, -346, 16, ...",...,Lulu,"[475, 475, 532, 771, 1046, 1288, 1534, 1776, 2...",KEITH,KogMaw,"[475, 475, 532, 766, 1161, 1438, 1776, 1936, 2...",Xpecial,Janna,"[515, 515, 583, 721, 870, 1059, 1205, 1342, 15...","['RekSai', 'Rumble', 'LeeSin']",http://matchhistory.na.leagueoflegends.com/en/...
4,NALCS,2015,Spring,Season,CLG,1,0,T8,35,"[40, 40, 44, -36, 113, 158, -121, -191, 23, 20...",...,Lulu,"[475, 475, 532, 807, 1042, 1338, 1646, 1951, 2...",Maplestreet8,Corki,"[475, 475, 532, 792, 1187, 1488, 1832, 2136, 2...",Dodo8,Annie,"[475, 475, 538, 671, 817, 948, 1104, 1240, 136...","['Rumble', 'Sivir', 'Rengar']",http://matchhistory.na.leagueoflegends.com/en/...


In [5]:
df.shape

(7620, 57)

In [6]:
df['Season'].unique()

array(['Spring', 'Summer'], dtype=object)

In [7]:
df['League'].unique()

array(['NALCS', 'EULCS', 'LCK', 'LMS', 'CBLoL', 'TCL', 'OPL', 'CLS',
       'LLN', 'LJL', 'LCL', 'WC', 'MSI', 'IEM', 'RR'], dtype=object)

In [8]:
df['blueTeamTag'].unique()

array(['TSM', 'CST', 'WFX', 'TIP', 'CLG', 'DIG', 'TL', 'C9', 'T8', 'GV',
       'TDK', 'NME', 'REN', 'FOX', 'IMT', 'NRG', 'NV', 'APX', 'P1', 'FLY',
       'EL', 'GIA', 'CW', 'SK', 'GMB', 'MYM', 'ROC', 'FNC', 'UOL', 'H2K',
       'OG', 'SPY', 'VIT', 'G2', 'S04', 'MSF', 'NIP', 'MM', 'CJE', 'JAG',
       'SKT', 'GET', 'NJE', 'IM', 'SSG', 'kt', 'ANC', 'SBENU', 'Longzhu',
       'SSB', 'KOO', 'LZ', 'NJF', 'CJ', 'AFs', 'EMF', 'ROX', 'KDM', 'ESC',
       'MVP', 'bbq', 'EEW', 'ahq', 'Crew', 'TPA', 'LOG', 'DLY', 'MSE',
       'NGU', 'DoR', 'HKES', 'yoeFW', nan, 'LGS', 'as', 'HKE', 'FW',
       'Mac', 'AS', 'M17', 'CGE', 'XG', 'JT', 'TM', 'FB', 'WS', 'RG',
       'HKA', 'AHQ', 'PNG', 'ITZ', 'CNB', 'OPK', 'VFK', 'KBM', 'RED',
       'g3x', 'BGG', 'KST', 'REM', 'TSW', 'PRG', 'ONE', 'BJK', 'TT', 'A',
       'ZONE', 'HWA', 'NR1', 'DP', 'BPI', 'OHM', 'CREW', 'SUP', 'AUR',
       'CLK', 'CRW', 'GS', 'GAL', 'P3P', 'INF', 'DW', 'LGC', 'SIN', 'TRI',
       'CHF', 'HLN', 'AV', 'AE', 'X5', 'RGC', 'ISG', 'H

In [9]:
df['Type'].value_counts()

Season           5653
Playoffs          775
International     658
Promotion         391
Regional          143
Name: Type, dtype: int64

In [10]:
df['League'].value_counts()

LCK      1445
NALCS    1272
EULCS    1099
LMS       778
TCL       653
OPL       458
WC        308
CBLoL     301
LCL       281
LJL       258
LLN       242
CLS       175
IEM       138
MSI       111
RR        101
Name: League, dtype: int64

In [11]:
df['gamelength'].mean()

37.01259842519685

In [12]:
df.columns

Index(['League', 'Year', 'Season', 'Type', 'blueTeamTag', 'bResult', 'rResult',
       'redTeamTag', 'gamelength', 'golddiff', 'goldblue', 'bKills', 'bTowers',
       'bInhibs', 'bDragons', 'bBarons', 'bHeralds', 'goldred', 'rKills',
       'rTowers', 'rInhibs', 'rDragons', 'rBarons', 'rHeralds', 'blueTop',
       'blueTopChamp', 'goldblueTop', 'blueJungle', 'blueJungleChamp',
       'goldblueJungle', 'blueMiddle', 'blueMiddleChamp', 'goldblueMiddle',
       'blueADC', 'blueADCChamp', 'goldblueADC', 'blueSupport',
       'blueSupportChamp', 'goldblueSupport', 'blueBans', 'redTop',
       'redTopChamp', 'goldredTop', 'redJungle', 'redJungleChamp',
       'goldredJungle', 'redMiddle', 'redMiddleChamp', 'goldredMiddle',
       'redADC', 'redADCChamp', 'goldredADC', 'redSupport', 'redSupportChamp',
       'goldredSupport', 'redBans', 'Address'],
      dtype='object')

In [13]:
df2 = df.drop('Address', 1)

In [14]:
df2.columns

Index(['League', 'Year', 'Season', 'Type', 'blueTeamTag', 'bResult', 'rResult',
       'redTeamTag', 'gamelength', 'golddiff', 'goldblue', 'bKills', 'bTowers',
       'bInhibs', 'bDragons', 'bBarons', 'bHeralds', 'goldred', 'rKills',
       'rTowers', 'rInhibs', 'rDragons', 'rBarons', 'rHeralds', 'blueTop',
       'blueTopChamp', 'goldblueTop', 'blueJungle', 'blueJungleChamp',
       'goldblueJungle', 'blueMiddle', 'blueMiddleChamp', 'goldblueMiddle',
       'blueADC', 'blueADCChamp', 'goldblueADC', 'blueSupport',
       'blueSupportChamp', 'goldblueSupport', 'blueBans', 'redTop',
       'redTopChamp', 'goldredTop', 'redJungle', 'redJungleChamp',
       'goldredJungle', 'redMiddle', 'redMiddleChamp', 'goldredMiddle',
       'redADC', 'redADCChamp', 'goldredADC', 'redSupport', 'redSupportChamp',
       'goldredSupport', 'redBans'],
      dtype='object')

In [15]:
df.head()

,League,Year,Season,Type,blueTeamTag,bResult,rResult,redTeamTag,gamelength,golddiff,...,redMiddleChamp,goldredMiddle,redADC,redADCChamp,goldredADC,redSupport,redSupportChamp,goldredSupport,redBans,Address
0,NALCS,2015,Spring,Season,TSM,1,0,C9,40,"[0, 0, -14, -65, -268, -431, -488, -789, -494,...",...,Fizz,"[475, 475, 552, 842, 1178, 1378, 1635, 1949, 2...",Sneaky,Sivir,"[475, 475, 532, 762, 1097, 1469, 1726, 2112, 2...",LemonNation,Thresh,"[515, 515, 577, 722, 911, 1042, 1194, 1370, 14...","['Tristana', 'Leblanc', 'Nidalee']",http://matchhistory.na.leagueoflegends.com/en/...
1,NALCS,2015,Spring,Season,CST,0,1,DIG,38,"[0, 0, -26, -18, 147, 237, -152, 18, 88, -242,...",...,Azir,"[475, 475, 552, 786, 1097, 1389, 1660, 1955, 2...",CoreJJ,Corki,"[475, 475, 532, 868, 1220, 1445, 1732, 1979, 2...",KiWiKiD,Annie,"[515, 515, 583, 752, 900, 1066, 1236, 1417, 15...","['RekSai', 'Janna', 'Leblanc']",http://matchhistory.na.leagueoflegends.com/en/...
2,NALCS,2015,Spring,Season,WFX,1,0,GV,40,"[0, 0, 10, -60, 34, 37, 589, 1064, 1258, 913, ...",...,Azir,"[475, 475, 533, 801, 1006, 1233, 1385, 1720, 1...",Cop,Corki,"[475, 475, 533, 781, 1085, 1398, 1782, 1957, 2...",BunnyFuFuu,Janna,"[515, 515, 584, 721, 858, 1002, 1168, 1303, 14...","['Leblanc', 'Zed', 'RekSai']",http://matchhistory.na.leagueoflegends.com/en/...
3,NALCS,2015,Spring,Season,TIP,0,1,TL,41,"[0, 0, -15, 25, 228, -6, -243, 175, -346, 16, ...",...,Lulu,"[475, 475, 532, 771, 1046, 1288, 1534, 1776, 2...",KEITH,KogMaw,"[475, 475, 532, 766, 1161, 1438, 1776, 1936, 2...",Xpecial,Janna,"[515, 515, 583, 721, 870, 1059, 1205, 1342, 15...","['RekSai', 'Rumble', 'LeeSin']",http://matchhistory.na.leagueoflegends.com/en/...
4,NALCS,2015,Spring,Season,CLG,1,0,T8,35,"[40, 40, 44, -36, 113, 158, -121, -191, 23, 20...",...,Lulu,"[475, 475, 532, 807, 1042, 1338, 1646, 1951, 2...",Maplestreet8,Corki,"[475, 475, 532, 792, 1187, 1488, 1832, 2136, 2...",Dodo8,Annie,"[475, 475, 538, 671, 817, 948, 1104, 1240, 136...","['Rumble', 'Sivir', 'Rengar']",http://matchhistory.na.leagueoflegends.com/en/...


In [16]:
df2 = df[['League', 'Type','blueTeamTag','bResult','rResult','redTeamTag','gamelength','golddiff']]

In [17]:
df2.columns

Index(['League', 'Type', 'blueTeamTag', 'bResult', 'rResult', 'redTeamTag',
       'gamelength', 'golddiff'],
      dtype='object')

In [18]:
df2.shape

(7620, 8)

In [19]:
df2.head()

,League,Type,blueTeamTag,bResult,rResult,redTeamTag,gamelength,golddiff
0,NALCS,Season,TSM,1,0,C9,40,"[0, 0, -14, -65, -268, -431, -488, -789, -494,..."
1,NALCS,Season,CST,0,1,DIG,38,"[0, 0, -26, -18, 147, 237, -152, 18, 88, -242,..."
2,NALCS,Season,WFX,1,0,GV,40,"[0, 0, 10, -60, 34, 37, 589, 1064, 1258, 913, ..."
3,NALCS,Season,TIP,0,1,TL,41,"[0, 0, -15, 25, 228, -6, -243, 175, -346, 16, ..."
4,NALCS,Season,CLG,1,0,T8,35,"[40, 40, 44, -36, 113, 158, -121, -191, 23, 20..."


In [20]:
df2['gamelength'].mean()

37.01259842519685

In [21]:
df2.dtypes

League         object
Type           object
blueTeamTag    object
bResult         int64
rResult         int64
redTeamTag     object
gamelength      int64
golddiff       object
dtype: object

In [22]:
df['League'].value_counts()

LCK      1445
NALCS    1272
EULCS    1099
LMS       778
TCL       653
OPL       458
WC        308
CBLoL     301
LCL       281
LJL       258
LLN       242
CLS       175
IEM       138
MSI       111
RR        101
Name: League, dtype: int64

In [23]:
# NALCS : Amérique du Nord
# LCK : Corée
# EULCS : Europe
# LMS : Taiwan Hong-Kong Macao
# TCL : Turquie
# OPL : Océanie
# WC : Worlds
# CBLoL : Brésil

df2 = df[['League', 'Type','blueTeamTag','bResult','rResult','redTeamTag','gamelength','golddiff']]

s=df2[(df2['League'] == 'NALCS') | 
      (df2['League'] == 'LCK') | 
      (df2['League'] == 'EULCS') |
      (df2['League'] == 'TCL') |
      (df2['League'] == 'CBLoL')]

In [24]:
s

,League,Type,blueTeamTag,bResult,rResult,redTeamTag,gamelength,golddiff
0,NALCS,Season,TSM,1,0,C9,40,"[0, 0, -14, -65, -268, -431, -488, -789, -494,..."
1,NALCS,Season,CST,0,1,DIG,38,"[0, 0, -26, -18, 147, 237, -152, 18, 88, -242,..."
2,NALCS,Season,WFX,1,0,GV,40,"[0, 0, 10, -60, 34, 37, 589, 1064, 1258, 913, ..."
3,NALCS,Season,TIP,0,1,TL,41,"[0, 0, -15, 25, 228, -6, -243, 175, -346, 16, ..."
4,NALCS,Season,CLG,1,0,T8,35,"[40, 40, 44, -36, 113, 158, -121, -191, 23, 20..."
...,...,...,...,...,...,...,...,...
7612,CBLoL,Season,PNG,1,0,KBM,34,"[0, 0, 34, -152, -422, -182, -367, -123, 165, ..."
7613,CBLoL,Season,KBM,1,0,PNG,30,"[0, 0, 37, 305, 716, 529, 739, 646, 1005, 1084..."
7614,TCL,Season,AUR,0,1,RBE,36,"[0, -8, -187, -37, -92, -164, -229, -424, -256..."
7615,TCL,Season,YC,0,1,SUP,34,"[0, 0, -18, -95, 45, -87, -117, 199, 126, 92, ..."


In [25]:
s['League'].unique()

array(['NALCS', 'EULCS', 'LCK', 'CBLoL', 'TCL'], dtype=object)

In [26]:
s['Type'].unique()

array(['Season', 'Playoffs', 'Regional', 'Promotion'], dtype=object)

In [27]:
x= s[(s['Type'] != 'Regional') & (s['Type'] != 'Promotion')]

In [28]:
x['Type'].unique()

array(['Season', 'Playoffs'], dtype=object)

In [29]:
x

,League,Type,blueTeamTag,bResult,rResult,redTeamTag,gamelength,golddiff
0,NALCS,Season,TSM,1,0,C9,40,"[0, 0, -14, -65, -268, -431, -488, -789, -494,..."
1,NALCS,Season,CST,0,1,DIG,38,"[0, 0, -26, -18, 147, 237, -152, 18, 88, -242,..."
2,NALCS,Season,WFX,1,0,GV,40,"[0, 0, 10, -60, 34, 37, 589, 1064, 1258, 913, ..."
3,NALCS,Season,TIP,0,1,TL,41,"[0, 0, -15, 25, 228, -6, -243, 175, -346, 16, ..."
4,NALCS,Season,CLG,1,0,T8,35,"[40, 40, 44, -36, 113, 158, -121, -191, 23, 20..."
...,...,...,...,...,...,...,...,...
7612,CBLoL,Season,PNG,1,0,KBM,34,"[0, 0, 34, -152, -422, -182, -367, -123, 165, ..."
7613,CBLoL,Season,KBM,1,0,PNG,30,"[0, 0, 37, 305, 716, 529, 739, 646, 1005, 1084..."
7614,TCL,Season,AUR,0,1,RBE,36,"[0, -8, -187, -37, -92, -164, -229, -424, -256..."
7615,TCL,Season,YC,0,1,SUP,34,"[0, 0, -18, -95, 45, -87, -117, 199, 126, 92, ..."


In [30]:
x['blueTeamTag'].value_counts()

kt       154
SKT      140
SSG      131
JAG      128
CLG      125
        ... 
CG         2
VK         2
OPT        2
SBENU      1
RBE        1
Name: blueTeamTag, Length: 104, dtype: int64

In [31]:
x['redTeamTag'].value_counts()

SKT      149
kt       139
JAG      134
TSM      129
SSG      125
        ... 
CG         2
100        1
SBENU      1
YC         1
GGS        1
Name: redTeamTag, Length: 104, dtype: int64

In [32]:
x.loc['redTeamTag'] = x['redTeamTag'].astype(object)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [34]:
x['redTeamTag'].dtype()

TypeError: 'numpy.dtype' object is not callable